# UCTP for HBA1

In [426]:
import gurobipy as gp
from gurobipy import GRB
import datetime

# Initialize the model
model = gp.Model("University Course Timetabling")


## Define Sets and Parameters

Sections

In [429]:
sections = list(range(1, 11))  # Sections 1 to 10

Dates and Periods

In [431]:
# Excluded dates
excluded_dates = [
    datetime.date(2024, 9, 30),
    datetime.date(2024, 10, 4),
    *[datetime.date(2024, 10, d) for d in range(14, 19)],  # October 14-18
    *[datetime.date(2024, 10, d) for d in range(22, 26)],  # October 22-25
    datetime.date(2024, 11, 21),
    datetime.date(2024, 11, 22),
    # Additional dates to exclude
    datetime.date(2024, 10, 30),
    datetime.date(2024, 10, 31),
    datetime.date(2024, 11, 1),
    datetime.date(2024, 12, 2)
]

In [432]:
# Generate dates from Sep 9 to Dec 6, excluding weekends
start_date = datetime.date(2024, 9, 9)
end_date = datetime.date(2024, 12, 6)
delta = datetime.timedelta(days=1)

dates = []
current_date = start_date
while current_date <= end_date:
    if (current_date.weekday() < 5 and
            current_date not in excluded_dates):
        dates.append(current_date)
    current_date += delta

periods = [1, 2, 3]  # Periods per day


In [433]:
# Define unavailable periods on specific dates
unavailable_periods = {
    datetime.date(2024, 9, 25): [3],
    datetime.date(2024, 11, 14): [2],
    datetime.date(2024, 11, 18): [3],
    datetime.date(2024, 11, 20): [3]
}

Courses and Required Sessions

In [435]:
# Courses and sessions
# Need to double-check is the number into optimization? For example, CME is removed since they are fixed.
courses = {
    'Communications': 25,
    'Finance': 25,
    'LPO': 25,
    'DMA': 24,
    'FinFun': 25,
    'SH': 2
}

Faculty

In [437]:
# Courses with faculty constraints
courses_with_faculty_constraints = ['Communications', 'Finance', 'LPO', 'DMA', 'FinFun']
courses_without_faculty_constraints = ['SH']

# Create faculties for each course
faculties_per_course = {}
faculty_sections_per_course = {}

for course in courses_with_faculty_constraints:
    faculties = [f'{course}_Faculty{i}' for i in range(1, 6)]
    faculties_per_course[course] = faculties
    # Assign sections to faculties
    faculty_sections = {}
    for idx, faculty in enumerate(faculties):
        sections_assigned = [idx * 2 + 1, idx * 2 + 2] 
        faculty_sections[faculty] = sections_assigned
    faculty_sections_per_course[course] = faculty_sections


## Define Decision Variables

In [439]:
x = model.addVars(
    sections,
    dates,
    periods,
    courses.keys(),
    vtype=GRB.BINARY,
    name="x"
)

## Objective Function

In [441]:
model.modelSense = GRB.MINIMIZE
model.setObjective(0)

## Constraints

Each Course's Sessions Must Be Scheduled

In [444]:
for s in sections:
    for c, sessions_required in courses.items():
        model.addConstr(
            gp.quicksum(x[s, d, p, c] for d in dates for p in periods) == sessions_required,
            name=f"SessionRequirement_s{s}_c{c}"
        )


No Overlapping Sessions for a Section

In [446]:
for s in sections:
    for d in dates:
        for p in periods:
            model.addConstr(
                gp.quicksum(x[s, d, p, c] for c in courses.keys()) <= 1,
                name=f"NoOverlap_s{s}_d{d}_p{p}"
            )


Faculty availability constraints across courses (excluding CME and SH)

In [448]:
# Faculty availability constraints per course
for course in courses_with_faculty_constraints:
    faculties = faculties_per_course[course]
    faculty_sections = faculty_sections_per_course[course]
    for faculty in faculties:
        assigned_sections = faculty_sections[faculty]
        for d in dates:
            for p in periods:
                model.addConstr(
                    gp.quicksum(
                        x[s, d, p, course]
                        for s in assigned_sections
                    ) <= 1,
                    name=f"FacultyNoOverlap_{course}_{faculty}_d{d}_p{p}"
                )

Constraints to prevent scheduling during unavailable periods

In [450]:
# Constraints to prevent scheduling during unavailable periods
for d, periods_unavailable in unavailable_periods.items():
    if d in dates: 
        for p in periods_unavailable:
            for s in sections:
                for c in courses.keys():
                    model.addConstr(
                        x[s, d, p, c] == 0,
                        name=f"UnavailablePeriod_s{s}_d{d}_p{p}_c{c}"
                    )

CME sessions are fixed

In [452]:
# CME sessions are fixed; prevent other courses from being scheduled at those times
# Define CME sessions for each group
cme_sessions = {
    'Group1': {
        'sections': [1, 2, 3],
        'sessions': [
            (datetime.date(2024, 9, 11), 3),
            (datetime.date(2024, 9, 23), 3),
            (datetime.date(2024, 10, 28), 3),
            (datetime.date(2024, 11, 15), 1),
            (datetime.date(2024, 11, 28), 3)
        ]
    },
    'Group2': {
        'sections': [4, 5, 6],
        'sessions': [
            (datetime.date(2024, 9, 9), 3),
            (datetime.date(2024, 9, 23), 1),
            (datetime.date(2024, 10, 28), 1),
            (datetime.date(2024, 11, 13), 3),
            (datetime.date(2024, 11, 27), 3)
        ]
    },
    'Group3': {
        'sections': [7, 8, 9, 10],
        'sessions': [
            (datetime.date(2024, 9, 11), 1),
            (datetime.date(2024, 9, 20), 1),
            (datetime.date(2024, 10, 29), 3),
            (datetime.date(2024, 11, 15), 3),
            (datetime.date(2024, 11, 26), 1)
        ]
    }
}

In [453]:
# Constraints to prevent scheduling other courses during CME sessions
for group_info in cme_sessions.values():
    sections_in_group = group_info['sections']
    for session_date, session_period in group_info['sessions']:
        if session_date in dates and session_period in periods:
            for s in sections_in_group:
                for c in courses.keys():  # All other courses
                    model.addConstr(
                        x[s, session_date, session_period, c] == 0,
                        name=f"CME_Overlap_s{s}_d{session_date}_p{session_period}_c{c}"
                    )

SH time and no overlapping constraints

In [455]:
sh1_dates = [d for d in dates if d < datetime.date(2024, 9, 23)]

# Define dates for SH2 (October 28 or October 29)
sh2_dates = [d for d in dates if d in [datetime.date(2024, 10, 28), datetime.date(2024, 10, 29)]]

# Enforce SH1 scheduling constraints
for s in sections:
    model.addConstr(
        gp.quicksum(x[s, d, p, 'SH'] for d in sh1_dates for p in periods) == 1,
        name=f"SH1_Scheduling_s{s}"
    )

# Enforce SH2 scheduling constraints
for s in sections:
    model.addConstr(
        gp.quicksum(x[s, d, p, 'SH'] for d in sh2_dates for p in periods) == 1,
        name=f"SH2_Scheduling_s{s}"
    )

In [456]:
# Define overlap constraints
overlap_constraints = [
    {
        'sh_sections': [1],
        'other_sections': [1, 2],
        'other_course': 'DMA'
    },
    {
        'sh_sections': [5, 6],
        'other_sections': [5, 6],
        'other_course': 'Finance'
    },
    {
        'sh_sections': [7, 8],
        'other_sections': [7, 8],
        'other_course': 'FinFun'
    },
    {
        'sh_sections': [9, 10],
        'other_sections': [9, 10],
        'other_course': 'LPO'
    }
]

# Add overlap constraints to the model
for oc in overlap_constraints:
    sh_sections = oc['sh_sections']
    other_sections = oc['other_sections']
    other_course = oc['other_course']
    for s in sh_sections:
        for d in dates:
            for p in periods:
                model.addConstr(
                    x[s, d, p, 'SH'] + gp.quicksum(x[os, d, p, other_course] for os in other_sections) <= 1,
                    name=f"Overlap_SH_{s}_{other_course}_{d}_{p}"
                )

## Solve the Model

In [458]:
# Optimize the model
model.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 6668 rows, 8640 columns and 28404 nonzeros
Model fingerprint: 0x8ed61d42
Variable types: 0 continuous, 8640 integer (8640 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


Display the Schedule

In [460]:
# Retrieve and display the schedule
if model.status == GRB.OPTIMAL:
    schedule = {}
    for s in sections:
        schedule[s] = []
        for d in dates:
            for p in periods:
                for c in courses.keys():
                    if x[s, d, p, c].X > 0.5:
                        schedule[s].append({
                            'date': d,
                            'period': p,
                            'course': c
                        })
    # Print the schedule for each section
    for s in sections:
        print(f"\nSchedule for Section {s}:")
        # Include CME sessions in the schedule
        cme_sessions_for_section = []
        for group_info in cme_sessions.values():
            if s in group_info['sections']:
                cme_sessions_for_section.extend([
                    {'date': session_date, 'period': session_period, 'course': 'CME'}
                    for session_date, session_period in group_info['sessions']
                ])
        # Combine scheduled sessions and CME sessions
        combined_schedule = schedule[s] + cme_sessions_for_section
        # Sort combined schedule
        combined_schedule.sort(key=lambda x: (x['date'], x['period']))
        for entry in combined_schedule:
            print(f"Date: {entry['date']}, Period: {entry['period']}, Course: {entry['course']}")
else:
    print("No feasible solution found.")


Schedule for Section 1:
Date: 2024-09-09, Period: 1, Course: LPO
Date: 2024-09-09, Period: 2, Course: FinFun
Date: 2024-09-09, Period: 3, Course: Communications
Date: 2024-09-10, Period: 1, Course: LPO
Date: 2024-09-10, Period: 2, Course: FinFun
Date: 2024-09-10, Period: 3, Course: DMA
Date: 2024-09-11, Period: 1, Course: Communications
Date: 2024-09-11, Period: 2, Course: Finance
Date: 2024-09-11, Period: 3, Course: CME
Date: 2024-09-12, Period: 1, Course: FinFun
Date: 2024-09-12, Period: 2, Course: Finance
Date: 2024-09-12, Period: 3, Course: DMA
Date: 2024-09-13, Period: 1, Course: Communications
Date: 2024-09-13, Period: 2, Course: SH
Date: 2024-09-13, Period: 3, Course: DMA
Date: 2024-09-16, Period: 1, Course: Finance
Date: 2024-09-16, Period: 2, Course: LPO
Date: 2024-09-16, Period: 3, Course: Communications
Date: 2024-09-17, Period: 1, Course: DMA
Date: 2024-09-17, Period: 2, Course: FinFun
Date: 2024-09-17, Period: 3, Course: FinFun
Date: 2024-09-18, Period: 1, Course: Communi